# 工具选择

工具使用支持一个称为`tool_choice`的参数，允许您指定希望Claude如何调用工具。在本笔记本中，我们将了解它的工作原理以及何时使用它。在进一步操作之前，请确保您熟悉与Claude一起使用工具的基础知识。

在使用`tool_choice`参数时，我们有三个可能的选项：

* `auto`允许Claude决定是否调用任何提供的工具
* `tool`允许我们强制Claude始终使用特定工具
* `any`告诉Claude它必须使用提供的工具之一，但不强制使用特定工具

让我们详细查看每个选项。我们将从导入Anthropic SDK开始：

In [31]:
from anthropic import Anthropic

client = Anthropic()
MODEL_NAME = "claude-sonnet-4-5"

## 自动

将`tool_choice`设置为`auto`允许模型自动决定是否使用工具。这是使用工具时的默认行为。

为了演示这一点，我们将为Claude提供一个虚假的网络搜索工具。我们将向Claude提问，其中一些问题需要调用网络搜索工具，而其他问题Claude应该能够自己回答。

让我们首先定义一个名为`web_search`的工具。请注意，为了保持此演示简单，我们实际上不在这里搜索网络：

In [137]:
def web_search(topic):
    print(f"pretending to search the web for {topic}")


web_search_tool = {
    "name": "web_search",
    "description": "A tool to retrieve up to date information on a given topic by searching the web",
    "input_schema": {
        "type": "object",
        "properties": {
            "topic": {"type": "string", "description": "The topic to search the web for"},
        },
        "required": ["topic"],
    },
}

接下来，我们编写一个函数，接受user_query并将其与`web_search_tool`一起传递给Claude。

我们还将`tool_choice`设置为`auto`：

```py
tool_choice={"type": "auto"}
```

这是完整的函数：

In [145]:
from datetime import date


def chat_with_web_search(user_query):
    messages = [{"role": "user", "content": user_query}]

    system_prompt = f"""
    Answer as many questions as you can using your existing knowledge.  
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
    """

    response = client.messages.create(
        system=system_prompt,
        model=MODEL_NAME,
        messages=messages,
        max_tokens=1000,
        tool_choice={"type": "auto"},
        tools=[web_search_tool],
    )
    last_content_block = response.content[-1]
    if last_content_block.type == "text":
        print("Claude did NOT call a tool")
        print(f"Assistant: {last_content_block.text}")
    elif last_content_block.type == "tool_use":
        print("Claude wants to use a tool")
        print(last_content_block)

让我们从一个Claude应该能够不使用工具就能回答的问题开始：

In [139]:
chat_with_web_search("What color is the sky?")

Claude did NOT call a tool
Assistant: The sky appears blue during the day. This is because the Earth's atmosphere scatters more blue light from the sun than other colors, making the sky look blue.


当我们问"天空是什么颜色？"时，Claude不使用工具。让我们尝试询问一些Claude应该使用网络搜索工具来回答的问题：

In [140]:
chat_with_web_search("Who won the 2024 Miami Grand Prix?")

Claude wants to use a tool
ToolUseBlock(id='toolu_staging_018nwaaRebX33pHqoZZXDaSw', input={'topic': '2024 Miami Grand Prix winner'}, name='web_search', type='tool_use')


当我们问"谁赢得了2024年迈阿密大奖赛？"时，Claude使用了网络搜索工具！

让我们尝试更多示例：

In [141]:
# Claude should NOT need to use the tool for this:
chat_with_web_search("Who won the superbowl in 2022?")

Claude did NOT call a tool
Assistant: The Los Angeles Rams won Super Bowl LVI in 2022, defeating the Cincinnati Bengals by a score of 23-20. The game was played on February 13, 2022 at SoFi Stadium in Inglewood, California.


In [144]:
# Claude SHOULD use the tool for this:
chat_with_web_search("Who won the superbowl in 2024?")

Claude wants to use a tool
ToolUseBlock(id='toolu_staging_016XPwcprHAgYJBtN7A3jLhb', input={'topic': '2024 Super Bowl winner'}, name='web_search', type='tool_use')


### 您的提示很重要！

在使用`auto`的`tool_choice`时，花时间编写详细的提示是很重要的。通常，Claude可能会过于热衷于调用工具。编写详细的提示有助于Claude确定何时调用工具，何时不调用。在上面的示例中，我们在系统提示中包含了特定说明：


```py
 system_prompt=f"""
    使用您的现有知识回答尽可能多的问题。
    仅对您无法自信回答的查询进行网络搜索。
    今天是 {date.today().strftime("%B %d %Y")}
    如果您认为用户的问题涉及尚未发生的未来事件，请使用搜索工具。
"""
```



## 强制使用特定工具

我们可以使用`tool_choice`强制Claude使用特定工具。在下面的示例中，我们定义了两个简单工具：
* `print_sentiment_scores` - 一个"诱使"Claude生成包含情感分析数据的结构良好的JSON输出的工具。有关此方法的更多信息，请参阅[使用Claude和工具使用提取结构化JSON](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/extracting_structured_json.ipynb)
* `calculator` - 一个非常简单的计算器工具，接受两个数字并相加

In [111]:
tools = [
    {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given tweet or piece of text.",
        "input_schema": {
            "type": "object",
            "properties": {
                "positive_score": {
                    "type": "number",
                    "description": "The positive sentiment score, ranging from 0.0 to 1.0.",
                },
                "negative_score": {
                    "type": "number",
                    "description": "The negative sentiment score, ranging from 0.0 to 1.0.",
                },
                "neutral_score": {
                    "type": "number",
                    "description": "The neutral sentiment score, ranging from 0.0 to 1.0.",
                },
            },
            "required": ["positive_score", "negative_score", "neutral_score"],
        },
    },
    {
        "name": "calculator",
        "description": "Adds two number",
        "input_schema": {
            "type": "object",
            "properties": {
                "num1": {"type": "number", "description": "first number to add"},
                "num2": {"type": "number", "description": "second number to add"},
            },
            "required": ["num1", "num2"],
        },
    },
]

我们的目标是编写一个名为`analyze_tweet_sentiment`的函数，该函数接受一条推文并打印该推文的基本情感分析。最终我们将"强制"Claude使用我们的情感分析工具，但首先我们将展示当我们**不**强制工具使用时会发生的什么。

在这个`analyze_tweet_sentiment`函数的第一个"坏"版本中，我们为Claude提供了两个工具。为了进行比较，我们将从将tool_choice设置为"auto"开始：

```py
tool_choice={"type": "auto"}
```

请注意，我们故意没有为Claude提供编写良好的提示，以便更容易看到强制使用特定工具的影响。

In [124]:
def analyze_tweet_sentiment(query):
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "auto"},
        messages=[{"role": "user", "content": query}],
    )
    print(response)

让我们看看当我们用推文"Holy cow, I just made the most incredible meal!"调用函数时会发生什么

In [125]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

ToolsBetaMessage(id='msg_staging_01ApgXx7W7qsDugdaRWh6p21', content=[TextBlock(text="That's great to hear! I don't actually have the capability to assess sentiment from text, but it sounds like you're really excited and proud of the incredible meal you made. Cooking something delicious that you're proud of can definitely give a sense of accomplishment and happiness. Well done on creating such an amazing dish!", type='text')], model='claude-sonnet-4-5', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=429, output_tokens=69))


Claude没有调用我们的情感分析工具：
> "很高兴听到这些！我实际上没有从文本中评估情感的能力，但听起来你对你制作的令人难以置信的饭菜真的很兴奋和自豪。

接下来，让我们想象有人发了这样的推文："我爱我的猫！我有四只，又收养了2只！猜猜我现在有多少只？"

In [128]:
analyze_tweet_sentiment(
    "I love my cats! I had four and just adopted 2 more! Guess how many I have now?"
)

ToolsBetaMessage(id='msg_staging_018gTrwrx6YwBR2jjhdPooVg', content=[TextBlock(text="That's wonderful that you love your cats and adopted two more! To figure out how many cats you have now, I can use the calculator tool:", type='text'), ToolUseBlock(id='toolu_staging_01RFker5oMQoY6jErz5prmZg', input={'num1': 4, 'num2': 2}, name='calculator', type='tool_use')], model='claude-sonnet-4-5', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=442, output_tokens=101))


Claude想要调用计算器工具：

> ToolUseBlock(id='toolu_staging_01RFker5oMQoY6jErz5prmZg', input={'num1': 4, 'num2': 2}, name='calculator', type='tool_use')

显然，当前的实现没有做我们想要的事情（主要是因为我们故意让它失败）。

接下来，让我们通过更新`tool_choice`强制Claude**始终**使用`print_sentiment_scores`工具：

```py
tool_choice={"type": "tool", "name": "print_sentiment_scores"}
```

除了将`type`设置为`tool`之外，我们还必须提供特定的工具名称。

In [132]:
def analyze_tweet_sentiment(query):
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "tool", "name": "print_sentiment_scores"},
        messages=[{"role": "user", "content": query}],
    )
    print(response)

现在，如果我们尝试用与之前相同的提示向Claude提问，它总是会调用`print_sentiment_scores`工具：

In [133]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

ToolsBetaMessage(id='msg_staging_018GtYk8Xvee3w8Eeh6pbgoq', content=[ToolUseBlock(id='toolu_staging_01FMRQ9pZniZqFUGQwTcFU4N', input={'positive_score': 0.9, 'negative_score': 0.0, 'neutral_score': 0.1}, name='print_sentiment_scores', type='tool_use')], model='claude-sonnet-4-5', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=527, output_tokens=79))


Claude调用了我们的`print_sentiment_scores`工具：

> ToolUseBlock(id='toolu_staging_01FMRQ9pZniZqFUGQwTcFU4N', input={'positive_score': 0.9, 'negative_score': 0.0, 'neutral_score': 0.1}, name='print_sentiment_scores', type='tool_use')

即使我们试图用"数学"推文难倒Claude，它仍然总是调用`print_sentiment_scores`工具：

In [134]:
analyze_tweet_sentiment(
    "I love my cats! I had four and just adopted 2 more! Guess how many I have now?"
)

ToolsBetaMessage(id='msg_staging_01RACamfrHdpvLxWaNwDfZEF', content=[ToolUseBlock(id='toolu_staging_01Wb6ZKSwKvqVSKLDAte9cKU', input={'positive_score': 0.8, 'negative_score': 0.0, 'neutral_score': 0.2}, name='print_sentiment_scores', type='tool_use')], model='claude-sonnet-4-5', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=540, output_tokens=79))


即使我们强制Claude调用我们的`print_sentiment_scores`工具，我们仍然应该采用一些基本的提示工程：

In [135]:
def analyze_tweet_sentiment(query):
    prompt = f"""
    Analyze the sentiment in the following tweet: 
    <tweet>{query}</tweet>
    """

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "auto"},
        messages=[{"role": "user", "content": prompt}],
    )
    print(response)

## 任意

`tool_choice`的最后一个选项是`any`，它允许我们告诉Claude"你必须调用一个工具，但你可以选择哪一个"。想象一下，我们想要使用Claude创建一个SMS聊天机器人。这个聊天机器人与用户"通信"的唯一方式是通过SMS文本消息。

在下面的示例中，我们制作了一个非常简单的文本消息助手，它可以使用两个工具：
* `send_text_to_user` 向用户发送文本消息
* `get_customer_info` 根据用户名查找客户数据

想法是创建一个始终调用这些工具之一且从不以非工具响应进行回复的聊天机器人。在所有情况下，Claude都应该通过尝试发送文本消息或调用`get_customer_info`来获取更多客户信息来回复。

最重要的是，我们将`tool_choice`设置为"any"：

```py
tool_choice={"type": "any"}
```

In [162]:
def send_text_to_user(text):
    # Sends a text to the user
    # We'll just print out the text to keep things simple:
    print(f"TEXT MESSAGE SENT: {text}")


def get_customer_info(username):
    return {
        "username": username,
        "email": f"{username}@email.com",
        "purchases": [
            {"id": 1, "product": "computer mouse"},
            {"id": 2, "product": "screen protector"},
            {"id": 3, "product": "usb charging cable"},
        ],
    }


tools = [
    {
        "name": "send_text_to_user",
        "description": "Sends a text message to a user",
        "input_schema": {
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "The piece of text to be sent to the user via text message",
                },
            },
            "required": ["text"],
        },
    },
    {
        "name": "get_customer_info",
        "description": "gets information on a customer based on the customer's username.  Response includes email, username, and previous purchases. Only call this tool once a user has provided you with their username",
        "input_schema": {
            "type": "object",
            "properties": {
                "username": {
                    "type": "string",
                    "description": "The username of the user in question. ",
                },
            },
            "required": ["username"],
        },
    },
]

system_prompt = """
All your communication with a user is done via text message.
Only call tools when you have enough information to accurately call them.  
Do not call the get_customer_info tool until a user has provided you with their username. This is important.
If you do not know a user's username, simply ask a user for their username.
"""


def sms_chatbot(user_message):
    messages = [{"role": "user", "content": user_message}]

    response = client.messages.create(
        system=system_prompt,
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "any"},
        messages=messages,
    )
    if response.stop_reason == "tool_use":
        last_content_block = response.content[-1]
        if last_content_block.type == "tool_use":
            tool_name = last_content_block.name
            tool_inputs = last_content_block.input
            print(f"=======Claude Wants To Call The {tool_name} Tool=======")
            if tool_name == "send_text_to_user":
                send_text_to_user(tool_inputs["text"])
            elif tool_name == "get_customer_info":
                print(get_customer_info(tool_inputs["username"]))
            else:
                print("Oh dear, that tool doesn't exist!")

    else:
        print("No tool was called. This shouldn't happen!")

让我们从简单的开始：

In [163]:
sms_chatbot("Hey there! How are you?")

=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: Hello! I'm doing well, thanks for asking. How can I assist you today?


Claude通过调用`send_text_to_user`工具来回复。

接下来，我们将向Claude询问一些更棘手的问题：

In [164]:
sms_chatbot("I need help looking up an order")

=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: Hi there, to look up your order details I'll need your username first. Can you please provide me with your username?


Claude想要发送一条文本消息，要求用户提供用户名。

现在，让我们看看当我们向Claude提供我们的用户名时会发生什么：

In [165]:
sms_chatbot("I need help looking up an order.  My username is jenny76")

=======Claude Wants To Call The get_customer_info Tool=======
{'username': 'jenny76', 'email': 'jenny76@email.com', 'purchases': [{'id': 1, 'product': 'computer mouse'}, {'id': 2, 'product': 'screen protector'}, {'id': 3, 'product': 'usb charging cable'}]}


Claude调用了`get_customer_info`工具，正如我们所希望的！

即使我们向Claude发送一条无意义的消息，它仍然会调用我们的工具之一：

In [166]:
sms_chatbot("askdj aksjdh asjkdbhas kjdhas 1+1 ajsdh")

=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: I'm afraid I didn't understand your query. Could you please rephrase what you need help with?
